In [1]:
# Useful starting lines
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
%load_ext autoreload
%autoreload 2

## Load the training data into feature matrix, class labels, and event ids:

In [2]:
from proj1_helpers import *
DATA_TRAIN_PATH = '../data/train.csv' # TODO: download train data and supply path here 
y, tX, ids = load_csv_data(DATA_TRAIN_PATH)
DATA_TEST_PATH = '../data/test.csv' # TODO: download train data and supply path here 
y_test, tX_test, ids_test = load_csv_data(DATA_TEST_PATH)

## Do your thing crazy machine learning thing here :) ...

### Explore the data

In [3]:
# Count nan values
# No Nan values !!! However there can be undefined values (see dataset documentation)
tX_nan_count = np.sum(np.where(tX == np.nan, 1, 0), axis = 0)
print("tX nan counts: "+str(tX_nan_count))

tX nan counts: [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]


In [42]:
a = np.asarray([[1,2,3],[4,5,6],[7,8,9]])
b = a
b

array([[1, 2, 3],
       [4, 5, 6],
       [7, 8, 9]])

In [46]:
np.median(a)

4.0

In [43]:
b[2,1]=2

In [44]:
print(a)

[[1 2 3]
 [4 5 6]
 [7 2 9]]


In [45]:
print(b)

[[1 2 3]
 [4 5 6]
 [7 2 9]]


In [4]:
# def getNanStatistics(tX,axis = 0):
#     print("Means: " + str(np.nanmean(tX,axis = axis)))
#     print("Standard devs: " + str(np.nanstd(tX,axis = axis)))
#     print("Max values: " + str(np.nanmax(tX,axis = axis)))
#     print("Min values: " + str(np.nanmin(tX,axis = axis)))

In [5]:
# def plot_feature_box_plots:
#     # TODO

In [6]:
np.corrcoef(tX,rowvar=False)

array([[ 1.00000000e+00, -4.55754982e-01,  1.68547985e-01,
         1.98764690e-01,  1.62660860e-01,  1.60523916e-01,
         1.62521066e-01,  2.28104685e-01,  4.58258755e-02,
         2.01464410e-01, -1.70725931e-02,  3.58962714e-01,
         1.62623001e-01,  1.31495478e-01,  2.57909175e-03,
        -6.17683359e-03,  9.94446173e-02,  1.00600077e-02,
        -7.44499050e-04, -2.33723570e-01,  7.41075286e-03,
         2.21984489e-01,  2.21077743e-01,  2.50158444e-01,
         2.47083378e-01,  2.47078209e-01,  1.62835955e-01,
         1.62613682e-01,  1.62609042e-01,  1.85371791e-01],
       [-4.55754982e-01,  1.00000000e+00,  1.90109488e-01,
        -2.49115929e-01, -1.76386051e-01, -1.90392025e-01,
        -1.75941781e-01,  4.32514093e-02,  1.77575270e-02,
        -1.46836679e-01,  3.49503614e-01, -4.19757335e-01,
        -1.76262284e-01, -1.45464145e-01, -2.10917148e-03,
         1.13208392e-03,  3.10647518e-01, -6.77706784e-03,
         3.40330242e-04,  1.83716377e-01, -1.59253407e-

In [7]:
# def box_plot(tX):
# # TODO: box plots to see correlations

In [8]:
# df = pd.read_csv(DATA_TRAIN_PATH)
# display(df)

In [9]:
# df.describe()

In [10]:
# getNanStatistics(tX)

## Pre-processing

In [11]:
feature_cols = ['DER_mass_MMC', 'DER_mass_transverse_met_lep',
       'DER_mass_vis', 'DER_pt_h', 'DER_deltaeta_jet_jet', 'DER_mass_jet_jet',
       'DER_prodeta_jet_jet', 'DER_deltar_tau_lep', 'DER_pt_tot', 'DER_sum_pt',
       'DER_pt_ratio_lep_tau', 'DER_met_phi_centrality',
       'DER_lep_eta_centrality', 'PRI_tau_pt', 'PRI_tau_eta', 'PRI_tau_phi',
       'PRI_lep_pt', 'PRI_lep_eta', 'PRI_lep_phi', 'PRI_met', 'PRI_met_phi',
       'PRI_met_sumet', 'PRI_jet_num', 'PRI_jet_leading_pt',
       'PRI_jet_leading_eta', 'PRI_jet_leading_phi', 'PRI_jet_subleading_pt',
       'PRI_jet_subleading_eta', 'PRI_jet_subleading_phi', 'PRI_jet_all_pt']
dtypes = 15 * ["float"] + ['float-angle'] + 2 * ['float'] + ['float-angle'] \
         + ['float'] + ['float-angle'] +['float'] + ['categorical'] + (2 * ['float']\
         + ['float-angle']) * 2 + ['float']

In [12]:
# # Test data preprocessor
# idx_col_cat = 22
# data_prep = DataPreprocessor(tX,feature_cols,dtypes)
# data_prep.replaceUndefVal(undef_new = np.nan)
# data_prep.nanStandardize()
# data_prep.replaceNanVal()
# tX_std = data_prep.getFeatures()

# tX_wo_undef_test = np.where(tX == -999.0, np.nan, tX)
# tX_wo_undef_test_std = nanStandardize(np.delete(tX_wo_undef_test,idx_col_cat,axis = 1))
# tX_wo_undef_test_std = np.where(np.isnan(tX_wo_undef_test_std), 0, tX_wo_undef_test_std)
# assert (np.delete(tX_std,idx_col_cat,axis = 1) == tX_wo_undef_test_std).all(),"Test failed"
# print("Test Ok")

In [13]:
# # One hot encode data
# data_prep_ohe = DataPreprocessor(tX,feature_cols,dtypes)
# data_prep_ohe.replaceUndefVal(undef_new = np.nan)
# data_prep_ohe.nanStandardize()
# data_prep_ohe.replaceNanVal()
# data_prep_ohe.oneHotEncode()
# tX_std_ohe = data_prep_ohe.getFeatures()

In [14]:
# multiHistPlots(tX_std,feature_cols, (15,15))

In [15]:
# print(tX_std_ohe[:,[29,30,31]])

In [16]:
# data_prep_ohe.cols

In [17]:
# tX_std_float = np.delete(tX_std,idx_col_cat,axis = 1)

In [18]:
# sel_feature_cols = ['DER_mass_MMC', 'DER_mass_transverse_met_lep',
#        'DER_mass_vis', 'DER_pt_h', 'DER_deltaeta_jet_jet', 'DER_mass_jet_jet',
#        'DER_prodeta_jet_jet', 'DER_deltar_tau_lep', 'DER_pt_tot', 'DER_sum_pt',
#        'DER_pt_ratio_lep_tau', 'DER_met_phi_centrality',
#        'DER_lep_eta_centrality', 'PRI_jet_num']

In [19]:
# def getColsIdx(cols):
#     feature_cols = ['DER_mass_MMC', 'DER_mass_transverse_met_lep',
#        'DER_mass_vis', 'DER_pt_h', 'DER_deltaeta_jet_jet', 'DER_mass_jet_jet',
#        'DER_prodeta_jet_jet', 'DER_deltar_tau_lep', 'DER_pt_tot', 'DER_sum_pt',
#        'DER_pt_ratio_lep_tau', 'DER_met_phi_centrality',
#        'DER_lep_eta_centrality', 'PRI_tau_pt', 'PRI_tau_eta', 'PRI_tau_phi',
#        'PRI_lep_pt', 'PRI_lep_eta', 'PRI_lep_phi', 'PRI_met', 'PRI_met_phi',
#        'PRI_met_sumet', 'PRI_jet_num', 'PRI_jet_leading_pt',
#        'PRI_jet_leading_eta', 'PRI_jet_leading_phi', 'PRI_jet_subleading_pt',
#        'PRI_jet_subleading_eta', 'PRI_jet_subleading_phi', 'PRI_jet_all_pt']
#     col_idx = np.arange(0,30)
#     feature_col_idx_dict = dict(zip(feature_col,col_idx))
#     if (type(cols) == str):
#         return feature_col_idx_dict.get(cols) 
#     elif (type(cols) == list):
#         return [feature_col_idx_dict.get(c) for c in cols]
#     else:
#         raise TypeError("The argument should be a list or a string")

In [20]:
# def nanStandardize(x):
#     mean = np.nanmean(x,axis = 0)
#     std = np.nanstd(x,axis = 0)
#     x =(x - mean) / std
#     return x

In [21]:
# def dropFeatures(tX,cols):
#     return tX[:,getColsIdx(cols)]

In [22]:
# def countUndefinedValues(tX,undef_value = -999.0):
#     return np.sum(tX == undef_value,axis = 0)

In [23]:
# def getRowsWithUndefIdx(tX,undef_value = -999.0):
#     undef_mat = np.where(tX == undef_value,1,0)
#     return np.where(np.sum(undef_mat, axis = 1)>0,True,False)

In [24]:
# tX_wo_undef = np.where(tX == -999.0, np.nan, tX)
# getNanStatistics(tX_wo_undef)

## Work with reduced tX

In [25]:
# tX_reduced = dropFeatures(tX,sel_feature_columns)

In [26]:
# # Check undefined values per column
# undef_val_count = countUndefinedValues(tX_reduced)
# print("Number of undefined values per column: " + str(undef_val_count))

In [27]:
# tX_reduced_wo_undef = np.where(tX_reduced == -999.0, np.nan, tX_reduced)

In [28]:
# getNanStatistics(tX_reduced_wo_undef)

## Preprocessing

In [93]:
from proj1_utils import *

In [99]:
cat_cols = [22]
x_train_num, x_train_cat = split_numerical_categorical(tX,cat_cols)
# Treat numerical values
x_train_num_nan = replace_undef_val_with_nan(x_train_num)
x_train_num_nan_std, train_mean, train_std = nan_standardize_fit(x_train_num_nan)
x_train_num_valid_std = replace_nan_val_with_median(x_train_num_nan_std)
# Treat categorical values
x_train_ohe_cat = one_hot_encode(x_train_cat)
x_train = np.hstack((add_bias(x_train_num_valid_std),x_train_ohe_cat))
# Treat labels
y_train = relabel_y_non_negative(y).reshape(-1,1)

In [101]:
cat_cols = [22]
x_test_num, x_test_cat = split_numerical_categorical(tX_test,cat_cols)
# Treat numerical values
x_test_num_nan = replace_undef_val_with_nan(x_test_num)
x_test_num_nan_std = nan_standardize_transform(x_test_num_nan,train_mean,train_std)
x_test_num_valid_std = replace_nan_val_with_median(x_test_num_nan_std)
# Treat categorical values
x_test_ohe_cat = one_hot_encode(x_test_cat)
x_test = np.hstack((add_bias(x_test_num_valid_std),x_test_ohe_cat))
# Treat labels
y_test = relabel_y_non_negative(y_test).reshape(-1,1)

## Import models

In [102]:
from implementations import *

## Ridge regression

In [136]:
lambda_ = 0.13
weights = ridge_regression(y_train,x_train,lambda_)

## Least squares

In [111]:
weights = least_squares(y_train,x_train)

## Generate predictions and save ouput in csv format for submission:

In [138]:
OUTPUT_PATH = '../output_submission/submission' # TODO: fill in desired name of output file for submission
y_pred = predict_labels(weights, x_test)
create_csv_submission(ids_test, y_pred, OUTPUT_PATH)

In [139]:
# Just to check
from sklearn.metrics import accuracy_score
print(accuracy_score(y_test,y_pred))

0.9641805018319789
